**Numerical approximation of the minimal invariant set of set-valued mapping**

We present a numerical program which approximate the boundary of a minimal invariant set of a specific set-valued mapping $F(x) = \overline{B_{\varepsilon}(f(x))}$, for some 2-D mapping $f$ and bound $\varepsilon > 0$. 

The approximation method uses the following procedures
- Approximate periodic points of the boundary points in some bounded region and compute their eigenvalues an eigenvectors.
- Numerical approximation of the 1-D unstable or stable manifold of saddle periodic points. This is done by applying boundary mapping or its inverse on sample points on the eigenvector of the periodic point corresponding to the targeted eigenvalue. 
- Some of the unstable manifolds represent part of the normal bundle of the boundary of a minimal invariant set; while for the stable manifold, they can represent boundary of some dual repeller.

In [1]:
def unstable_mani_brute(vec,indicator,eigensearch,dual,eigenstack,period):
    
    '''
    find unstable manifold of unstable period points as approximation of MIS by brute force
    (take number of points on the unstable linear eigenspace of periodic points and map forward)
    vec         - an nxd array containing n perioidic points of the system
    indicator   - an 1-D array containing indices for unstable points
    eigensearch - an 2-D array containing direction of search for unstable points
    dual        - indices of dual repeller boundary points
    
    '''
    
    num_cores = multiprocessing.cpu_count()
    fast = 1 # faster computation by taking sample on negative and positive
    traj_each = {}
    perturb_tol = 1e-5 # distance of samples from equilibrium
    mistol = 1e-6 # minimum distance to record 
    inputs = tqdm(range(len(indicator))) # for parallel programming of invariant manifold approx
    
    # start approximating unstable manifold from unstable periodic points
    if __name__ == "__main__":
        processed_list = Parallel(n_jobs=num_cores)(delayed(inv_mani)(i,indicator[i],period,dual,eigenstack,fast,perturb_tol,mistol) for i in inputs)
        
    # non-parallel    
#    traj_final = np.array([])
#     len1 = 0
#     for k in range(len(processed_list)):
#         len1 += len(processed_list[k])
#         #if k == 0:
#         #    traj_final = np.array(processed_list[k])
#         #else:
#         #    traj_final = np.append(traj_final,processed_list[k],axis=0)
#         #traj_each[k] = np.shape(traj_final)[0]
#         traj_each[k] = len1

    traj_final = processed_list
    return traj_final#,traj_each

def inv_mani(indicator_counter,i,period,dual,eigenstack,fast,perturb_tol,mistol):
    n_period = int(period[i]) # periodicity of the targeted point
    if any(i==dual):
        if any((np.abs(eigenstack[i])<1)&(eigenstack[i]<0)): # detect if need doubling period to make positive eigen
            n_period *= 2
        F_inner = lambda xx : F_inv(xx)
        print('inverse map used')
        n = 20000 # total points used in tangent directions
        if n_period > 6:
            nn = 10*n_period # total iterations
        else:
            nn = 30*n_period # total iterations
    else:
        if any((np.abs(eigenstack[i])>1)&(eigenstack[i]<0)): # detect if need doubling period to make positive eigen
            n_period *= 2
        F_inner = lambda xx : F(xx)
        n = 10000 # total points used in tangent directions
        if n_period > 6:
            nn = 10*n_period # total iterations
        else:
            nn = 500*n_period # total iterations
            if any(np.abs(np.abs(eigenstack[i])-1)<5e-2):
                nn = nn = 2000*n_period # total iterations
    if fast: # both side of the eigendirection
        vec_iter = vec[i]+perturb_tol*eigensearch[indicator_counter]*np.reshape(np.linspace(-1,1,n),(-1,1))
    else: # only one side and uses double map
        vec_iter = vec[i]+perturb_tol*eigensearch[indicator_counter]
        vec_iter_next = F_inner(vec_iter) # use two iteration instead of from vec
        vec_iter = vec_iter + (vec_iter_next - vec_iter)*np.reshape(np.linspace(0,1,n),(-1,1))
    traj_final = np.array(vec_iter)
    check = 1
    j = 0
    while j < nn:
        vec_iter = np.transpose(F_inner(np.transpose(vec_iter)))
        if check:
            if np.linalg.norm(vec_iter[-1,:]-vec[i])>mistol:
                check = 0
        traj_final = np.append(traj_final,vec_iter,axis=0)
        j += 1
        #if np.any(np.abs(vec_iter[:,:2])>coord_lim): # stop iteration when coord lim reached
            #    break#
            
            
    if not(fast): # the other side of the eigendirection
        vec_iter = vec[i]-perturb_tol*eigensearch[indicator_counter]
        vec_iter_next = F_inner(vec_iter)
        vec_iter = vec_iter + (vec_iter_next - vec_iter)*np.reshape(np.linspace(0,1,n),(-1,1))
        check = 1
        j = 0
        while j < nn:
            vec_iter = np.transpose(F_inner(np.transpose(vec_iter)))
            if check:
                if np.linalg.norm(vec_iter[-1,:]-vec[i])>mistol:
                    check = 0
            traj_final = np.append(traj_final,vec_iter,axis=0)
            j += 1
    return traj_final

In [2]:
def unstable_mani(vec,indicator,eigensearch,dual,eigenstack,period):
    '''
    find unstable manifold of unstable period points as approximation of MIS as one array (traj_vec) or all trajectories (traj_all)
    vec         - an nxd array containing n perioidic points of the system
    indicator   - an 1-D array containing indices for unstable points
    eigensearch - an 2-D array containing direction of search for unstable points
    dual        - indices of dual repeller boundary points
    
    '''
    
    d = 2 # dimension of problem
    if np.size(indicator) == 0: # no invariant manifold to approximate
        try:
            indicator[0]
        except IndexError:
            print('no unstable points to approximate')
    global mp_on # so that this can be used in other function
    mp_on = 0 # whether to turn on the arbitrary precision
    inputs = tqdm(range(len(indicator))) # for parallel programming of invariant manifold approximation
    if mp_on:
        import mpmath as mp
        mp.dps = 20 # precision of array
        processed_list = [inv_mani_better(vec,i,indicator[i],dual,period,eigensearch,eigenstack) for i in inputs]
    else:
        import mpmath as mp
        del mp # remove the mpmath so that parallel programming can work        
        num_cores = multiprocessing.cpu_count() # number of cpu cores 
        # start approximating unstable manifold from unstable periodic points
        if __name__ == "__main__":
            processed_list = Parallel(n_jobs=num_cores)(delayed(compute_inv_mani)(vec,i,indicator[i],dual,period,\
                                                                             eigensearch,eigenstack) for i in inputs)   
    traj_vec = np.array(processed_list,dtype=object)
    print('Shape of trajectory vector: {}'.format(np.shape(traj_vec)))
    return traj_vec

def compute_inv_mani(vec,indicator_counter,i,dual,period,eigensearch,eigenstack):
    traj_all = {}
    traj_vec = np.array([])
    traj_each = np.array([]) # to record each trajectory point
    traj_compare = np.array([]) # to detect closing of boundary of minimal invariant set 
    #coord_lim = 200 # limiting coordinates
    global perturb_tol, mistol_forward, mistol_dual, stable_tol, conv_tol,\
    inner_max, time_allowed, maxpoint #tolerence
    cross_on, compare_other = 0, 0 # compare with self or other computed boundary points

    print('vec '+str(i)) # maxpoint is the max points in between an iteration
    
    # Define maximum iteration for each case for different periodicity
    n_period = int(period[i])
    if n_period > 8:
        perturb_tol = 1e-9
        max_iter = 200
    elif n_period > 4:
        max_iter = 8
    elif n_period > 2:
        max_iter = 500
    elif n_period <= 2:
        max_iter = 8000
    
    # Reduce max iteration if it is on dual repeller
    eigenn = eigenstack[i]
    if np.any(indicator_counter==dual):
        max_iter = 8
#         maxpoint = 1e8 #increase the limit

    # Define forward or backward continuation of unstable invariant manifold
    if np.any(indicator_counter==dual):
        if np.any((np.abs(eigenstack[i])<1)&(eigenstack[i]<0)): # detect if need doubling period to make positive eigen
            n_period *= 2
        def Finv_period_n(xx,n):
            for g in range(n):
                xx = F_invn(xx,n_period)
            return xx
        F_inner_period_n = lambda xx,n : Finv_period_n(xx,n)
        mistol = mistol_dual
        print('inverse map used')
    else:
        if np.any((np.abs(eigenstack[i])>1)&(eigenstack[i]<0)): # detect if need doubling period to make positive eigen
            n_period *= 2
        def F_period_n(xx,n):
            for g in range(n):
                xx = F_n(xx,n_period)
            return xx
        F_inner_period_n = lambda xx,n : F_period_n(xx,n)
        mistol = mistol_forward
    
    for pm in range(2): # two direction, minus and plus from tangent direction
        kkk = 0 # for recording all trajectory
        traj_all[str(pm)] = {}
        vec_0 = np.array(vec[i])
        if pm == 0:
            vec_0 += perturb_tol*eigensearch[indicator_counter]            
        else:
            vec_0 -= perturb_tol*eigensearch[indicator_counter]
        vec_1 = F_inner_period_n(vec_0,1) # n_period steps of extended map
        if mp_on:
            dist_vec_0 = [mp.mpf(i) for i in vec_1-vec_0]
        else:
            dist_vec_0 = np.array(vec_1-vec_0)
#         print('distance between original first iteration = {}'.format(np.linalg.norm(dist_vec_0))) # for debug
        traj_add = np.array([vec_0,vec_1])
        if len(traj_each) == 0:
            traj_each = np.array([vec_0])
        else:
            traj_each = np.append(traj_each,[vec_0],axis=0)

        # define distance between iterations
        dist_diff = np.linalg.norm(vec_1-vec_0)*1e-3
        self_trigger, dist_stable, dist_cross, self_cross, self_compare = 0, 1, 1, 1, 100 # parameters for self crossing
        if np.any(indicator_counter==dual):
            compare_other = 0 # no compare for dual repeller
        traj_all[str(pm)][str(kkk)] = np.array([vec_1]) # first trajec
        vec_iter_old = np.array(vec_1)
        exceed = 0 # track whether time exceeded 
        inf_loop = 0 # track whether too much loop
        j = 1 # track iteration number
        param_inner = np.array([0.,1.]).reshape(2,1) # parameter 0 to 1
        
        # run until iteration points converge or converged to stable point and other stopping criterion
        while (abs(dist_diff)>=conv_tol or j<30 or np.isnan(dist_diff)) and dist_stable>stable_tol and self_cross>mistol/3 and dist_cross > mistol/3:
            if exceed == 1 and exceed_break and not(np.any(indicator_counter==dual)): # break if time exceeded once for not dual
                break
            if inf_loop == 1 and inf_loop_break: # break if time exceeded once
                break
            vec_iter = F_inner_period_n(vec_iter_old,1) # n steps of forward extended map
            traj_each = np.append(traj_each,[vec_iter],axis=0) # record each trajectory
            if np.linalg.norm(vec_iter-vec_iter_old)>1e-2: # switch on self cross if sufficiently far away from fixed point
                self_trigger = 1
            dist_diff = np.linalg.norm(vec_iter-vec_iter_old) # redefine distance between iter
            if np.any(indicator_counter==dual):
                if len(indicator_unstable)>0: # redefine distance with unstable points if any
                    dist_stable = min(np.min(np.linalg.norm(vec_iter[:]-vec[indicator_unstable,:],axis=1)),dist_stable) # (2d-1)-D compare
            else:
                if len(indicator_stable)>0: # redefine distance with stable points if any
                    dist_stable = min(np.min(np.linalg.norm(vec_iter[:]-vec[indicator_stable,:],axis=1)),dist_stable) # (2d-1)-D compare
            k = 1 # track number of iterations
            add_pt = []
            if dist_diff > mistol or np.isnan(dist_diff): # add points between iterations if points too far or it is infinity
                add_pt = np.array([vec_iter_old,vec_iter]) # initialise points to add
                if not(np.any(indicator_counter==dual) and use_previous_param): # whether to use previous parameters for dual repeller approximation
                    param_inner = np.array([0.,1.]).reshape(2,1) # parameter 0 to 1
                elif len(param_inner) > 2:
                    add = F_inner_period_n((vec_0+np.kron(param_inner[1:-1],dist_vec_0).reshape(len(param_inner[1:-1]),len(dist_vec_0),order='F'))
                              .transpose(),j).transpose()
                    add_pt = np.insert(add_pt,[1 for t in range(np.shape(add)[0])],add,axis=0)
                    if not(cut_coordlim): # delete points over threshold for continuation
                        delete_pt = np.where(np.logical_or(np.abs(add_pt)>coord_lim,np.isnan(add_pt)))[0]
                        if len(delete_pt) != 0:
                            add_pt = np.delete(add_pt,delete_pt,axis=0)
                            param_inner = np.delete(param_inner,delete_pt,axis=0)
                if len(add_pt)==0: # exit when no point to add
                    print('no more point to add at j = {}'.format(j))
                    break
                if tol_twodim == 1:
                    find_ind = np.where(np.logical_and(np.linalg.norm(add_pt[1:,:2]-add_pt[:-1,:2],axis=1)>mistol,
                                       np.linalg.norm(add_pt[1:,:2]-add_pt[:-1,:2],axis=1)<mistol_upper))[0] # find where to add points 
                else:
                    find_ind = np.where(np.logical_and(np.linalg.norm(add_pt[1:,:]-add_pt[:-1,:],axis=1)>mistol,
                                       np.linalg.norm(add_pt[1:,:]-add_pt[:-1,:],axis=1)<mistol_upper))[0] # find where to add points 
                start_time1 = time.time()
                while len(find_ind)>0: # keep adding points 
                    param_add = np.array(param_inner[find_ind]+(param_inner[find_ind+1]-param_inner[find_ind])/2) # add parameters
                    add = F_inner_period_n((vec_0+np.kron(param_add,dist_vec_0).reshape(len(param_add),len(dist_vec_0),order='F')) # add points from the very start
                              .transpose(),j).transpose() # since the function F require column vectors
                    
                    # detect whether points are over coordinates limit
                    if np.any(np.abs(np.reshape(add,-1))>coord_lim) and cut_coordlim: 
                        print('over threshold inner')
                        print(vec_iter,vec_iter_old,j,k)
                        break
                    elif np.any(indicator_counter==dual): # delete portion of the add point which exceed threshold for dual repeller approximation
                        delete_pt = np.where(np.abs(add)>coord_lim)[0]
                        if len(delete_pt) != 0:
                            find_ind = np.delete(find_ind,delete_pt)
                            add = np.delete(add,delete_pt,axis=0)
                            param_add = np.delete(param_add,delete_pt)
                            if len(add) == 0: # no more points within coord to add
                                break
                    add_pt = np.insert(add_pt,find_ind+1,add,axis=0) # update points to add
                    param_inner = np.insert(param_inner,find_ind+1,param_add.reshape([len(param_add),1]),axis=0) # update parameters
                    find_ind = np.where(np.logical_and(np.linalg.norm(add_pt[1:,:]-add_pt[:-1,:],axis=1)>mistol,
                                       np.linalg.norm(add_pt[1:,:]-add_pt[:-1,:],axis=1)<mistol_upper))[0] # update where to add
                    #print('vector {}, at {} iterations and inner {} with {} points added on {}'.format(i,j,k,len(add_pt),np.round(vec_iter,3))) # for debug
                    k += 1
                    
                    # detect infinite adding of points
                    if k>inner_max: 
                        print('inner infinite loop')
                        print('max distance = {} at vec {}'.format(np.max(np.linalg.norm(add_pt[1:,:]-add_pt[:-1,:],axis=1)),i))
                        inf_loop = 1
                        if only_success == 1: # only add the points which satisfy tolerance
                            add_pt = add_pt[:find_ind[0]+1,:] # only add the points which satisfy tolerance
                        k = -1
                        break

                    #check time in the loop, break if exceed time
                    if time.time()-start_time1 > time_allowed:
                        print('exceed time at '+str(j)+' on vec '+str(i))
                        print('max distance = {} at vec {}'.format(np.max(np.linalg.norm(add_pt[1:,:]-add_pt[:-1,:],axis=1)),i))
                        exceed = 1
                        if only_success == 1: # only add the points which satisfy tolerance
                            add_pt = add_pt[:find_ind[0]+1,:]
                        break


                # for comparing distance with stable points
                if np.any(indicator_counter==dual):
                    for kk in indicator_unstable:
                        dist_stable = min(np.min(np.linalg.norm(add_pt[:,:]-vec[kk,:],axis=1)),dist_stable) # (2d-1)-D compared
                else:
                    for kk in indicator_stable:
                        dist_stable = min(np.min(np.linalg.norm(add_pt[:,:]-vec[kk,:],axis=1)),dist_stable) # (2d-1)-D compared
                    
                # for comparing distance with other boundary points
                if compare_other:
                    if (np.linalg.norm(add_pt[:,:d]-vec[i,:d],axis=1)>mistol).all() and len(traj_compare)!=0:
                        for compare in range(len(add_pt)):
                            dist_cross = min(dist_cross,np.min(np.linalg.norm(add_pt[compare,:d]-traj_compare,axis=1)))
                            if dist_cross < mistol/3:
                                add_pt = np.array(add_pt[:compare+1,:]) # break loop when manifold touches other manifold
                                print('manifold cross at {}'.format(add_pt[compare,:]))
                                break
                                
                # for detecting self crossing
                if cross_on and (np.shape(traj_add)[0] > self_compare) and self_trigger and dist_cross > mistol/3: # detecting self cross
                    for compare in range(len(add_pt)):
                        self_cross = min(self_cross,np.min(np.linalg.norm(add_pt[compare,:d]-traj_add[:-self_compare+1,:d],axis=1)))
                        if self_cross <= mistol/3:
                            add_pt = np.array(add_pt[:compare+1,:]) # break loop when manifold touches itself
                            print('self cross at {}'.format(add_pt[compare,:d]))
                            break

                    #additional tests for internal cross within one iteration
                    if self_cross > mistol and len(add_pt)>self_compare*2 and dist_cross > mistol:
                        for compare in range(self_compare*2,len(add_pt)):
                            self_cross = min(self_cross,np.min(np.linalg.norm(add_pt[compare,:d]-add_pt[1:compare-self_compare+1,:d],axis=1)))
                            if self_cross <= mistol/3:
                                add_pt = np.array(add_pt[:compare+1,:]) # break loop when manifold touches itself
                                print('self self cross at {}'.format(add_pt[compare,:d]))
                                break

                traj_add = np.insert(traj_add,np.shape(traj_add)[0],add_pt[1:,:],axis=0) # update trajectory points
                kkk += 1
                traj_all[str(pm)][str(kkk)] = np.array(add_pt[1:,:])
            else:
                traj_add = np.insert(traj_add,np.shape(traj_add)[0],vec_iter,axis=0)
                kkk += 1
                traj_all[str(pm)][str(kkk)] = np.array([vec_iter])

            vec_iter_old = np.array(vec_iter) # update the endpoint vector

            j += 1
            if j > max_iter: # stop before max iter
                print('maximum {} iteration reached'.format(max_iter))
                break
            
            if len(traj_add)>maxpoint: # stop when max points reached
                print('enough points (>{}) at vec {}'.format(maxpoint,i))
                break

        if abs(dist_diff)<conv_tol:
            print('traj converged')
        elif dist_stable<=stable_tol:
            print('converged to stable point')
        elif cross_on and self_cross<=mistol/3:
            print('self crossed')
        elif dist_cross <= mistol/3:
            print('crossed with other traj')
        elif exceed == 1 and exceed_break and not(np.any(indicator_counter==dual)):
            print('exceed threshold time')
        elif j <= max_iter:
            print('something else made me converge')

        if len(traj_vec)==0: # update final boundary trajectory points
            traj_vec = np.array(traj_add)
            traj_compare = np.array(traj_add[:,:2])
        else:
            traj_vec = np.insert(traj_vec,np.shape(traj_vec)[0],traj_add,axis=0)
            traj_compare = np.insert(traj_compare,0,traj_add[:,:2],axis=0)
        
    return [traj_vec,traj_all]

print('done')

done


In [3]:
'''definitions in finding fixed and periodic points to outward normal equation (extended system) and 
approximating boundary'''
import sympy as sp
import numpy as np
from numpy import sin, cos
import time
import scipy.optimize as opt
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import sys
start_time = time.time()

# parameters
root_tolerance, tolerance, root_tolerance_1= 1e-10, 1e-7, 1e-6
rotation1 = np.array([[0,1],[-1,0]])

# define functions, jacobians
a, b, x, y, n1, n2, eps = sp.symbols('a b x y n1 n2 eps')
c,d,k = sp.symbols('c d k')
func_sym = [1+y-a*x**2,b*x] # symbolic function of deterministic function
func_sym_jacinv = (sp.Matrix(func_sym).jacobian([x,y])**-1).T # InvTransp jacobian symbolic
func_norm = func_sym_jacinv*sp.Matrix([n1,n2]) # InvTransp jacobian multiply by n

# Jacobian of determimnistic map
jac_det = sp.lambdify([(x,y,a,b,eps)],sp.Matrix(func_sym).jacobian([x,y])) # jacobian for map lambda function

# inverse
func_sym_inv = [y/b,x+a/b**2*y**2-1] # Inverse function
func_inv = sp.lambdify([(x,y,a,b,eps)],func_sym_inv) # inverse lambda function

# extended mapping
func_norm_unit = [func_norm[0]/(func_norm[0]**2+func_norm[1]**2)**(1/2),
                  func_norm[1]/(func_norm[0]**2+func_norm[1]**2)**(1/2)]
x1 = [func_sym[0]+eps*func_norm_unit[0],func_sym[1]+eps*func_norm_unit[1]] # One step of extended map
F_sym = sp.Array([x1[0],x1[1],func_norm_unit[0],func_norm_unit[1]])
F_func = sp.lambdify([(x,y,n1,n2,a,b,eps)],F_sym,modules={'atan': np.arctan})
FFdet_sym = sp.Array([func_sym[0],func_sym[1]]) # deterministic without normal
FFdet_func = sp.lambdify([(x,y,a,b,eps)],FFdet_sym,modules={'atan': np.arctan})
Fdet_sym = sp.Array([func_sym[0],func_sym[1],func_norm_unit[0],func_norm_unit[1]])#determnistic
Fdet_func = sp.lambdify([(x,y,n1,n2,a,b,eps)],Fdet_sym,modules={'atan': np.arctan})

# inverse of extended mapping
xx = [x-eps*n1,y-eps*n2]
x1 = func_inv([xx[0],xx[1],a,b,eps])
func_sym_inv_norm = (sp.Matrix(func_sym_inv).jacobian([x,y])**-1).T # InvTrans jacobian of inverse symbolic
func_sym_inv_norm = func_sym_inv_norm.subs([(x,xx[0]),(y,xx[1])])
func_inv_norm = [func_sym_inv_norm[0,0]*n1+func_sym_inv_norm[0,1]*n2,
    func_sym_inv_norm[1,0]*n1+func_sym_inv_norm[1,1]*n2] # InvTrans jacobian of inverse multiply by n func
F_inv_sym = sp.Array([x1[0],x1[1],func_inv_norm[0]/(func_inv_norm[0]**2+func_inv_norm[1]**2)**(1/2),\
                      func_inv_norm[1]/(func_inv_norm[0]**2+func_inv_norm[1]**2)**(1/2)])
F_inv_func = sp.lambdify([(x,y,n1,n2,a,b,eps)],F_inv_sym)

# Jacobian of extended map
jac_sym = sp.Matrix(F_sym).jacobian([x,y,n1,n2]) # jacobian for map symbolic
jac = sp.lambdify([(x,y,n1,n2,a,b,eps)],jac_sym) # jacobian for map lambda function
jac_inv_sym = sp.Matrix(F_inv_sym).jacobian([x,y,n1,n2]) # jacobian for inverse map symbolic
jac_inv = sp.lambdify([(x,y,n1,n2,a,b,eps)],jac_inv_sym) # jacobian for inverse map lambda function


# Defines functions (the input is of size 4xn or 2xn)
def ff(xx): #deterministic without normal
    x,y = xx[0], xx[1]
    return np.array(FFdet_func([x,y,a_sub,b_sub,epsilon]))
def ff_n(xx,n): # n-steps of deterministic mapping
    for i in range(n):
        xx = ff(xx)
    return xx
def f(xx): # deterministic mapping
    x, y, n1, n2 = xx[0], xx[1], xx[2], xx[3]
    return np.array(Fdet_func([x,y,n1,n2,a_sub,b_sub,epsilon]))
def f_n(xx,n): # n-steps of deterministic mapping
    for i in range(n):
        xx = f(xx)
    return xx
def F(xx): # extended map with a_sub and b_sub
    x, y, n1, n2 = xx[0], xx[1], xx[2], xx[3]
    return np.array(F_func([x,y,n1,n2,a_sub,b_sub,epsilon]))
def F_2_n(xx,n): # 2n-steps of extended
    for i in range(n):
        xx = F(F(xx))
    return xx
def F_n(xx,n): # n-steps of extended
    for i in range(n):
        xx = F(xx)
    return xx
def F_eq(xx): # stable fixed point equation
    return F(xx)-xx
def F_eq2(xx): # two steps of extended periodic equation
    return F(F(xx))-xx
def Jac(xx): # Jacobian of the extended map
    x, y, n1, n2 = xx[0], xx[1], xx[2], xx[3]
    return jac([x,y,n1,n2,a_sub,b_sub,epsilon])
def Jac_2(xx):
    return np.matmul(Jac(F(xx)),Jac(xx))
def Jac_2_eq(xx):
    return Jac_2(xx)-np.eye(4)
def Jac_eqn(xx,n): # Jacobian of n times extended map
    jac1 = np.eye(4)
    nn = 0
    while nn < n:
        jac1 = np.matmul(Jac(xx),jac1) # chain rule
        nn+=1
        xx = F(xx)
    return jac1-np.eye(4) # Jac of periodic points equation

# Jacobian of deterministic map
def Jac_det(xx):
    x, y= xx[0], xx[1]
    return jac_det([x,y,a_sub,b_sub,epsilon])
def Jac_detn(xx,n):
    jac1 = np.eye(2)
    nn = 0
    while nn < n:
        jac1 = np.matmul(Jac_det(xx),jac1) # chain rule
        nn+=1
        xx = F(xx)
    return jac1

# Inverse functions 
def f_inv(xx): # deterministic mapping
    x, y = xx[0], xx[1]
    return np.array(func_inv([x,y,a_sub,b_sub,epsilon]))
def F_inv(xx): # inverse boundary map with a_sub and b_sub
    x, y, n1, n2 = xx[0], xx[1], xx[2], xx[3]
    return np.array(F_inv_func([x,y,n1,n2,a_sub,b_sub,epsilon]))
def F_inv2(xx): # two steps of extended period equation
    return F_inv(F_inv(xx))-xx
def F_invn(xx,n): # n periodic equation 
    xx1 = xx
    for i in range(n):
        xx = F_inv(xx)
    return xx
def F_inv_eq(xx): # unstable fixed point equation
    return F_inv(xx)-xx
def F_inv_eq2(xx): # periodic point equation with period 2
    return np.array(F_inv2(xx)-xx)
def Jac_inv(xx): # Jacobian of the inverse extended map
    x, y, n1, n2 = xx[0], xx[1], xx[2], xx[3]
    return jac_inv([x,y,n1,n2,a_sub,b_sub,epsilon])
def Jac_inv2(xx):
    return np.matmul(Jac_inv(F_inv(xx)),Jac_inv(xx))-np.eye(4)
def jac_inv_eqn(xx,n):
    jac1 = np.eye(4)
    while n > 0:
        xx1 = F_invn(xx,n-1)
        jac1 = np.matmul(Jac_inv(xx1),jac1) # chain rule
        n-=1
    return jac1-np.eye(4)

# defintion of function for n_period
def F_period_n(xx,n):
    for i in range(n):
        xx = F_n(xx,n_period)
    return xx

# iteration inverse function
def Finv_period_n(xx,n):
    for i in range(n):
        xx = F_invn(xx,n_period)
    return xx

def Finv_period(xx):
    xx = F_invn(xx,n_period)

# search for periodic points on a square grid by hybrid method
def find_period(n_period, sample_grid, normal_grid, root_tolerance, tolerance, coord_lim_grid):
    vec = []
    n = sample_grid
    nn = n**2*normal_grid
    d = 2 # dimension of problem
    if n_period != 1:
        F_periodeq = lambda xx: F_n(xx,n_period)-xx
        Jac_periodeq = lambda xx: Jac_eqn(xx,n_period)
    else:
        F_periodeq = lambda xx: F(xx)-xx
        Jac_periodeq = lambda xx: Jac(xx)-np.eye(2*d)
    xt, yt, zt = np.linspace(coord_lim_grid[0,0],coord_lim_grid[0,1],n), np.linspace(coord_lim_grid[1,0],coord_lim_grid[1,1],n),\
    np.linspace(coord_lim_grid[2,0],coord_lim_grid[2,1],normal_grid)
    xx, yy, zz = np.meshgrid(xt,yt,zt)
    xx, yy, n11, n22 = np.reshape(xx,nn), np.reshape(yy,nn), cos(np.reshape(zz,nn)), sin(np.reshape(zz,nn))
    
    for i in range(nn):
        if i%50 == 0:
            print('searching periodic points .. {}/{} ..'.format(i,nn))
        sol = opt.root(F_periodeq, [xx[i], yy[i], n11[i], n22[i]], jac=Jac_periodeq, method='hybr', tol=root_tolerance)
        if np.linalg.norm(F_periodeq(sol.x))<root_tolerance_1:
            if len(vec)==0: # when vec is empty
                vec.append(sol.x)
                F_sol = sol.x # add the image as well if it has period n_period
                for i in range(1,n_period): # add the periodic points if not found previously 
                    F_sol = F(F_sol)
                    if all(np.linalg.norm(F_sol-np.array(vec),axis=1)>tolerance): # remove close enough points
                        vec.append(F_sol)
                    else: # points already found so are their periodic counterparts
                        break
            else: 
                # add the periodic points if not found previously
                F_sol = sol.x
                if all(np.linalg.norm(F_sol-np.array(vec),axis=1)>tolerance): # remove close enough points    
                    vec.append(sol.x)
                    F_sol = sol.x # add the image as well if it has period n_peiod
                    for i in range(1,n_period): # add the periodic points if not found previously 
                        F_sol = F(F_sol)
                        if all(np.linalg.norm(F_sol-np.array(vec),axis=1)>tolerance): # remove close enough points
                            vec.append(F_sol)                        
                        else:
                            break
    return np.array(vec)

def compacting(i,vec):
    max_iter = int(100)
    n_period = 1
    vec_iter = np.array(vec[i])
    pertb = vec_iter[d:]
    vec_iter[:d] += 1e-2*np.matmul(rotation1,pertb)
    if any(np.isnan(F_n(vec_iter,max_iter))): # exceed numerical limit
        return i
    elif any(np.abs(F_n(vec_iter,max_iter))>coord_lim): # exceed desired limit
        return i
    else: # the other tangent direction
        vec_iter[:d] -= 2e-2*np.matmul(rotation1,pertb)
        if any(np.isnan(F_n(vec_iter,max_iter))):
            return i
        elif any(np.abs(F_n(vec_iter,max_iter))>coord_lim):
            return i
        
# numerical continuation for finding periodic points        
def find_period_continue(vec,vec1,n,tolerance,period_new,n_period,ii,near_tol): # find periodic points in grid
    rotation1 = np.array([[0,1],[-1,0]]) # rotate from normal to tangent
    pertb = vec[ii,d:]
    target_array = np.array(vec[ii,:]) # tangent line test for other periodic points
    target_array[:d] -= searchtol*np.matmul(rotation1,pertb) 
    target_add = 2*np.array(searchtol*np.matmul(rotation1,pertb)/n)
    test_any = 0 # whether to find any fresh periodic points
    for i in range(n+2):
        if i == n+1:
            target_array = np.array(vec[ii,:]) # test original points
        theta_angle = 0
        for j in range(n+2):
            if j > 0:
                target_array[d:]= np.array([cos(theta_angle),sin(theta_angle)]) # test different angles
            sol = opt.root(F_periodeq, target_array, jac=Jac_periodeq, method='hybr', tol=root_tolerance)
            #print('original point is {}, with iteration is {}'.format(vec[ii,:],sol.x)) # for debug
            if np.linalg.norm(F_periodeq(sol.x))<tolerance and (np.linalg.norm(sol.x-vec[ii,:])<near_tol or test_any): # only add point if it is periodic point
                if len(vec1)==0:
                    vec1.append(sol.x)
                    # add period
                    for k in range(1,n_period+1):
                        if n_period%k==0:
                            if np.linalg.norm(F_n(sol.x,k)-sol.x)<tolerance:
                                period_new = np.append(period_new,k)
                                break
                    F_sol = sol.x # add the image as well if it has period n_period
                    for i in range(1,n_period):
                        F_sol = F(F_sol)

                        if all(np.linalg.norm(F_sol-np.array(vec1),axis=1)>tolerance):
                            vec1.append(F_sol)
                            period_new = np.append(period_new,k)
                            #print('inner at {}'.format(np.shape(vec1)[0])) # for debug
                        else:
                            break
                else:
                    F_sol = sol.x

                    if all(np.linalg.norm(F_sol-np.array(vec1),axis=1)>tolerance):
                        vec1.append(sol.x)
                        # add period
                        for k in range(1,n_period+1):
                            if n_period%k==0:
                                if np.linalg.norm(F_n(sol.x,k)-sol.x)<tolerance:
                                    period_new = np.append(period_new,k)
                                    break
                        # add the image as well if it has period n_period
                        F_sol = sol.x
                        for i in range(1,n_period):
                            F_sol = F(F_sol)

                            if all(np.linalg.norm(F_sol-np.array(vec1),axis=1)>tolerance):
                                vec1.append(F_sol)
                                period_new = np.append(period_new,k)
                                #print('inner at {}'.format(np.shape(vec1)[0]))
                            else:
                                break
            if j > 0:
                theta_angle += 2*np.pi/n
        target_array[:d] += np.array(target_add)
        return vec1,period_new


end_time = time.time()
print('elapsed time after definition is {}'.format(end_time-start_time))

elapsed time after definition is 1.6226592063903809


In [8]:
'''start of approximating manifold'''
from numpy import pi, cos, sin
import scipy.linalg as sl
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
jj = 0 # initialise iteration for continuation
total_plus= 0 # placeholder
start_time = time.time()

# Define constants and parameters
a_sub, b_sub, epsilon = 1.12, 0.3, 0.0625
tol_twodim = 1 # whether to optimize points to two dimensional only
exceed_break = 1 # whether to stop the program when exceeded time for non dual
inf_loop_break = 0 # whether to stop the program when infinite loop
only_success = 0 # whether to include points with correct tolerance
cut_coordlim = 0 # whether to break when one point exceed limit along the invariant manifold approximation
use_previous_param = 1 # whether to use previous parameter for dual
mistol_upper = 10 # to ignore anything more than this distance away
ult_lim = 1e1000 # ultimate coordinate limit
coord_lim = 1e500 # limit on state space
d = 2 # dimension of problem
perturb_tol, mistol_forward, mistol_dual, stable_tol, conv_tol,\
inner_max, time_allowed, maxpoint = 1e-5, 2e-4, 5e-3, 1e-14, 1e-19, 2000, 10, 7e5 #tolerence

# Some other settings of the approximation
brute = 0 # whether to use brute force shooting method
draw_mani = 1 # whether to compute unstable manifold or not
dualoff, compact = 0,1 # whether to turn off dual computation or only compact set is return
sample_grid = 20 # number of grid in each dimension for x-y plane
normal_grid = 20 # number of grid for the third dimension
coord_lim_grid = np.array([[-2,2],[-2,2],[0,2*np.pi]]) # coordinates of grid to search in
coord_lim_grid1 = np.array(coord_lim_grid) # can define different coordinate limits
coord_lim_grid_all = {} # initialise general coordinate limit
test_period = np.int64([1]) # which periodicity to search
for i in test_period: # different search grid for different periods
    if i == 12:
        coord_lim_grid_all[i] = coord_lim_grid1
    else:
        coord_lim_grid_all[i] = coord_lim_grid
num_cores = multiprocessing.cpu_count() # for parallel programming cpu number of cores

if jj == 0: # initialise the data array if this is the first iteration (not numerical continuation)
    eigen_all, eigenvec_all, vec_all, manifold_all, indicator_all, indicator_stable_all = {}, {}, {}, {}, {}, {}
    eigen_det_all, traj_each_all, dual_all, indicator_unstable_all = {}, {}, {}, {}
    vec_det_all, period_all= {}, {} # deterministic vector and period
    record_all = {}
    a_start, b_start, eps_start = float(a_sub), float(b_sub), float(epsilon) # define the start of parameters 

# Deterministic steady point
n = 150
nn = n**2
xt, yt = np.linspace(-10,10,n), np.linspace(-10,10,n)
xx, yy = np.meshgrid(xt,yt)
xx, yy = np.reshape(xx,nn), np.reshape(yy,nn)
ll = np.unique(np.round(ff_n(np.array([xx,yy]),1000),10),axis=1)
ll = ll[:,np.unique(np.where(np.isfinite(ll))[1])]
if np.any(np.abs(ll)>coord_lim):
    vec_det_all[str(jj)] = [] # empty if exceed coord limit
else:
    vec_det_all[str(jj)] = ll


# parallel programming for finding periodic points
period = []
inputs = tqdm(test_period)
if __name__ == "__main__":
    processed_list = Parallel(n_jobs=num_cores)(delayed(find_period)(i, sample_grid, normal_grid,\
        root_tolerance, tolerance,coord_lim_grid_all[i]) for i in inputs)
vec = np.array([])
original_find = processed_list

# get rid of duplicates and assign period
for k in range(len(processed_list)):
    target = np.array(processed_list[k])
    
    if len(target) == 0: # no periodic point
        continue
    elif len(vec) != 0:
        delete_ind = []
        if len(np.shape(target))==1: # one vector only
            length = 1
            if np.linalg.norm(vec-target) < tolerance: #duplicates
                continue
        else:
            length = np.shape(target)[0]
            for kk in range(length):
                if any(np.linalg.norm(vec-target[kk],axis=1) < tolerance): #duplicates
                    delete_ind.append(kk)
            target = np.delete(target,delete_ind,axis=0)
    
    # check if period correct
    temp_period = []
    delete_ind = []
    if len(np.shape(target))==1:
        target1 = np.array(target)
        for iters in range(1,test_period[k]+1):
            if test_period[k]%iters != 0: # not a factor of periodicity
                continue
            target2 = F_n(target1,iters)
            if np.linalg.norm(target1-target2)<1e-3:
                if iters != test_period[k]: # get rid of points with wrong periodicity
                    delete_ind.append(0)
                else:
                    temp_period.append(iters)
                break
            if iters == test_period[k]:
                temp_period.append(iters)
    else:
        for ii in range(np.shape(target)[0]):
            target1 = np.array(target[ii,:])
            for iters in range(1,test_period[k]+1):
                if test_period[k]%iters != 0: # not a factor of periodicity
                    continue
                target2 = F_n(target1,iters)
                if np.linalg.norm(target1-target2)<1e-3:
                    if iters != test_period[k]: # get rid of points with wrong periodicity
                        delete_ind.append(ii)
                    else:
                        temp_period.append(iters)
                    break
                if iters == test_period[k]:
                    temp_period.append(iters)
    period = np.append(period,np.int64(temp_period))
    target = np.delete(target,delete_ind,axis=0)
    
    
    if (len(vec)==0)&(len(target)>0): # first assignment of vector
        if len(np.shape(target))==1:
            vec = [target]
        else:
            vec = target
    elif len(target) == 0: # no periodic point
        continue
    else:
        if len(np.shape(target))==1:
            vec = np.append(vec,np.array([target]),axis=0)
        else:
            vec = np.append(vec,np.array(target),axis=0)
    print(len(target),test_period[k])

vec = np.array(vec)
end_time = time.time()
print('elapsed time after searching periodic points: {}s'.format(np.round(end_time-start_time,2)))

# getting rid points not on compact invariant set
if compact == 1:
    delete_ind = np.array([])
    inputs = tqdm(range(np.shape(vec)[0]))
    if __name__ == "__main__":
        processed_list = Parallel(n_jobs=num_cores)(delayed(compacting)(i,vec) for i in inputs)
    delete_ind = [i for i in processed_list if i !=None]
    if len(delete_ind)>0:
        vec = np.delete(vec,np.int64(delete_ind),0)
        period = np.delete(period,np.int64(delete_ind))

# record eigenvalues
eigenstack_det = np.zeros((np.shape(vec)[0],d),dtype=complex)
eigenstack = np.zeros((np.shape(vec)[0],2*d-1),dtype=complex)
eigenvecstack = np.zeros((np.shape(vec)[0],2*d,2*d-1),dtype=complex)
for i in range(np.shape(vec)[0]):
    n_period = int(period[i])
    Jac_period = lambda xx: Jac_eqn(xx,n_period)+np.eye(4)
    Jac_period_det = lambda xx: Jac_detn(xx,n_period)
    eigen, eigenvec = sl.eig(Jac_period(vec[i,:]))
    eigen_det = sl.eigvals(Jac_period_det(vec[i,:]))
    
    # remove zero eigenvalue (reduce dimension to 2d-1)
    remove_ind = np.where(np.abs(eigen)<1e-5)[0]
    if len(remove_ind)==0:
        print('no_zero')
        remove_ind = 2*d-1
    eigenstack_det[i,:]=eigen_det
    if len(remove_ind)>1: # one eigenvalue very small near 0
        print('more than one 0 eigenvalues')
        remove_ind = 3
    eigenstack[i,:]=np.delete(eigen,remove_ind)
    eigenvecstack[i,:,:]=np.delete(eigenvec,remove_ind,1)

# remove points with two or more unstable eigenspace and more than limit
if dualoff:
    print('deleting points')
    delete_ind = np.array([])
    for i in range(np.shape(vec)[0]):
        count = 0
        if any(abs(vec[i,:d])>coord_lim):
            delete_ind=np.insert(delete_ind,0,i)
            continue
        for m in range(np.size(eigenstack[i,:])):
            if np.abs(eigenstack[i,m])>1:
                count += 1
        if count > 1:
            delete_ind=np.insert(delete_ind,0,i)


    vec = np.delete(vec,np.int64(delete_ind),0)
    eigenstack = np.delete(eigenstack,np.int64(delete_ind),0)
    eigenvecstack = np.delete(eigenvecstack,np.int64(delete_ind),0)
    eigenstack_det = np.delete(eigenstack_det,np.int64(delete_ind),0)
    period = np.delete(period,np.int64(delete_ind),0)
    
np.set_printoptions(suppress=True)
print('number of periodic points: {}'.format(np.shape(vec)[0]))
period_all[str(jj)] = np.array(period)
vec_all[str(jj)] = np.array(vec) # define fist periodic points
eigen_det_all[str(jj)] = eigenstack_det
eigen_all[str(jj)] = eigenstack
eigenvec_all[str(jj)] = eigenvecstack
print('periodic points: {}'.format(np.round(np.transpose(vec),2)))
print('eigenvalues: {}'.format(np.round(np.transpose(eigenstack),7)))
print('period: {}'.format(period))

# determine periodic points
lenvec = np.shape(vec)[0]
record_vec = [0]
for counter in range(lenvec-1):
    if np.linalg.norm(F(vec[counter,:])-vec[counter+1,:])>1e-3:
        record_vec.append(counter+1) # record start of periodic points
record_all[str(jj)] = record_vec

# determine stable and unstable points
indicator_unstable = [] # indices of all unstable points
dual = [] # indices of 1-D stable
indicator = [] # indices of 1-D unstable
indicator_stable = [] # # indices of stable
eigensearch = [] # eigenvector for direction of search
i = 0
while i < np.shape(vec)[0]:
    eigenhere = np.array(eigenstack[i])
    eigenvechere = np.array(eigenvecstack[i])
    if all(np.abs(eigenhere)>1): # all eigen diverge
        indicator_unstable.append(i)
        i += 1
        continue
    elif all(np.abs(eigenhere)<=0.999): # all eigen converge
        indicator_stable.append(i)
        i += 1
        continue
    choice = np.argwhere(np.abs(eigenhere)>0.999)
    if len(choice)>d-1: # more than d-1 unstable direction which must be on dual since in dim d, all points on boundary attracts outer points
        target=eigenvechere[:,np.argwhere(np.abs(eigenhere)<=0.999)[0][0]]
        dual.append(len(indicator))
    else:
        target=eigenvechere[:,choice[0][0]] # choose the unstable eigenvalue direction
    eigensearch.append(target)
    indicator.append(i)
    i += 1

eigensearch = np.real(np.array(eigensearch))
indicator = np.array(indicator)
dual = np.array(dual)
if np.size(indicator) == 0:
    sys.exit('no unstable points to approximate')
indicator_all[str(jj)] = indicator
indicator_stable_all[str(jj)] = indicator_stable
dual_all[str(jj)] = dual
indicator_unstable_all[str(jj)] = indicator_unstable

end_time = time.time()
print('elapsed time before unstable manifold calc: {}s'.format(np.round(end_time-start_time,2)))

#Approximate invariant manifold

if draw_mani:
    if not(brute):
        manifold_all[str(jj)] = unstable_mani(vec,indicator,eigensearch,dual,eigenstack,period)
    else:
        manifold_all[str(jj)] = unstable_mani_brute(vec,indicator,eigensearch,dual,eigenstack,period)
end_time = time.time()
print('elapsed time in unstable manifold calc: {}s'.format(np.round(end_time-start_time,2)))



100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 832.20it/s]


4 1
elapsed time after searching periodic points: 2.01s


100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]


number of periodic points: 3
periodic points: [[-1.27  0.71  0.65]
 [-0.4   0.28  0.13]
 [-0.95  0.17 -0.18]
 [-0.32  0.99 -0.98]]
eigenvalues: [[-28.8722166+0.j  -1.7836773+0.j  -1.6221618+0.j]
 [  2.9499358+0.j   0.1694818+0.j   0.1831323+0.j]
 [ -0.1021721+0.j  -0.0950182+0.j  -0.112894 +0.j]]
period: [1. 1. 1.]
elapsed time before unstable manifold calc: 2.08s


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2999.50it/s]


Shape of trajectory vector: (3, 2)
elapsed time in unstable manifold calc: 24.65s


In [9]:
#2d/3d better plot
%matplotlib qt
import random
d2 = 1 # dimension 2 or 3
if not(d2):
    ax = plt.axes(projection='3d')
start_time = time.time()
plot_all = 0 # whether to plot everything at once
save_work = 0 # whether to save the data point as npy file
for jj in range(1):
# jj = 0 # which iteration (for continuation)
    a_sub = np.round(a_start+jj/nn*total_plus,5) # take the a parameter value 
    m_size, m_size_vec = 1, 5 # size of the plot
    deter = 0 # whether to plot deterministic point
    plotperiodicity = 1 # whether to display periodicity of periodic points
    plot_nec_vec = 1 # only plot necessary periodic points
    plotvec = 0 # whether to plot periodic points
    ploteach = 0 # whether to plot each manifold as curve in plotperiodicity
    plotimage = 0 # whether plot image of manifold
    plotnext = 0 # whether to plot next traj for dual
    xz = 0 # whether to plot angle versus x
    add_minus = 0 # whether to add minus theta to the same 3dplot
    limit = 5 # limit of dual data
    minustheta = 0 # 3-dimensional add minus theta
    individual_period, target_period_all, no_all = 1 , [1] , [0,1] # to plot individual unstable manifolds
    period_tgt = 1 # whether to plot same periodic points together
    clear_interior = 0 # clear interior and only display boundary in individual period
    round_no = 30 # number of dp to round
    epsilon_tol = 1e-6 # tolerance of the epsilon compansate for clearing interior
    manicolor1, veccolor1 = '#1f77b4', 'red' # color of manifold and vector for minimal attractor
    manicolor_dual, veccolor_dual = 'orange', 'purple' # color of manifold and vector for dual repeller

    # take data from the approximation
    vec = vec_all[str(jj)]
    dual = dual_all[str(jj)]
    vec_det = vec_det_all[str(jj)]
    period = period_all[str(jj)]
    indicator = indicator_all[str(jj)]
    indicator_stable = indicator_stable_all[str(jj)]
    if deter: # for deterministic mapping
        plt.scatter(vec_det[0,:],vec_det[1,:],s=40,color='blue') # plot determinitic steady points

    if individual_period: # plot for specific periodicity of periodic points
        boundary, boundary_dual = np.array([]), np.array([]) # to combine data from MIS and dual
        count = 0 # count for periodic points
        data = manifold_all[str(jj)]
        for target_period in target_period_all: # find periodic points with target periodicity
            for no in no_all:
                count += 1
                counter = 0 # counter for which target point in no_all           
                k = 0
                datadual = [] # for recording data
                true_ind = [] # recording indices of target data
                for ii in range(len(data)): # go over number of periodic points
            #         print(period[indicator[ii]]) # for debugging
                    if (period[indicator[ii]] == target_period):
                        counter += 1
                        if period_tgt: # plot periodic points together
                            plot_bool = (counter > no*target_period)&(counter <= (no+1)*target_period)
                        else:
                            plot_bool = (counter > no)&(counter <= (no+1))
                        if  plot_bool:
                            if np.any(ii==dual):
                                manicolor, veccolor = manicolor_dual, veccolor_dual #color of manifold and vector #'#1f77b4'
                                isdual = 1
                            else:
                                manicolor, veccolor = manicolor1, veccolor1
                                isdual = 0
                            k+=1
                            true_ind.append(ii)
                            for jjj in range(0,2): # left and right unstable manifold
                                krg = len(data[ii][1][str(jjj)]) # can be changed if not all part of invariant manifold wanted to be plotted
                                datadual2 = [] # for ploteach
                                for kk in range(0,krg):
                                    datadual1 = data[ii][1][str(jjj)][str(kk)]
                                    if isdual:
                                        datadual1 = np.delete(datadual1,np.unique(np.where(np.abs(datadual1)>limit)[0]),axis=0) # delete points exceeding limit
                                        datadual1 = np.delete(datadual1,np.unique(np.where(np.isnan(datadual1))[0]),axis=0) #delete nan points
                                    if len(datadual) == 0:
                                        datadual = np.array(datadual1)
                                    elif len(datadual2) == 0:
                                        datadual2 = np.array(datadual1)
                                    else:
                                        if len(np.shape(datadual)) == 1:
                                            datadual = np.insert(np.array([datadual]),1,datadual1,axis=0)
                                            datadual2 = np.insert(np.array([datadual2]),1,datadual1,axis=0)
                                        else:
    #                                         plt.plot(datadual[:,0],datadual[:,1])
                                            datadual = np.insert(datadual,np.shape(datadual)[0],datadual1,axis=0)
                                            datadual2 = np.insert(datadual2,np.shape(datadual2)[0],datadual1,axis=0)
                                if ploteach: # plot each traj if it is dual
                                    if d2 == 1:
                                        if jjj == 1:
                                            plotno = 2170000 # plot only part of the unstable manifold for clarity
                                        else:
                                            plotno = 2170000
                                        plt.plot(datadual2[:plotno,0],datadual2[:plotno,1],markersize=m_size,color=manicolor)
                                    else:
                                        plt.plot(datadual2[:,0],datadual2[:,1],
                                     np.arctan2(datadual2[:,3],datadual2[:,2]),
                                     color=manicolor,markersize=m_size)                 
                if len(datadual) == 0:
                    print('empty')
                    continue

                # delete identical copies
                datadual = np.unique(np.round(datadual,round_no),axis=0)

                # plot one step inverse step next
                if plotnext: 
                    Ft = F_invn(np.transpose(datadual),target_period);
                    if d2==1:
                        plt.plot(Ft[0,:],Ft[1,:],'.',color='cyan',markersize=m_size)
                    else:
                        plt.plot(Ft[0,:],Ft[1,:],np.arctan2(Ft[3,:],Ft[2,:]),'.',color='black',markersize=m_size)

                # compute the image
                if period_tgt:
                    image_full = ff(datadual.transpose())
                    datadual11 = F(datadual.transpose()).transpose()
                else:
                    if isdual:
                        datadual11 = F(datadual.transpose()).transpose()
                        image_full = ff(datadual.transpose())
                    else:
                        datadual11 = F_n(datadual.transpose(),target_period-1).transpose()
                        image_full = ff(datadual11.transpose())

                # only display boundary for mis or dual
                if clear_interior and not(plot_all):
                    data_len, image_len = np.shape(datadual.transpose()[:2,:])[1], np.shape(image_full)[1]
                    now=time.time()
                    if not(isdual):  # for mis
                        def cutt(datvec,datadual,image_full):
                            if np.any(np.linalg.norm(datadual[datvec,:2]-image_full.transpose(),axis=1)<epsilon-epsilon_tol):
                                return datvec
                        print('cutting')
                        inputs = tqdm(range(np.shape(datadual)[0]))
                        if __name__ == "__main__":
                            cut_ind= Parallel(n_jobs=num_cores)(delayed(cutt)(i,datadual,image_full) for i in inputs)
                    else: # for dual
                        def cutt(datvec,datadual,image_full):
                            if np.any(np.linalg.norm(datadual[:,:2]-image_full[:,datvec].transpose(),axis=1)<epsilon-epsilon_tol):
                                return datvec
                        print('cutting')
                        inputs = tqdm(range(np.shape(datadual)[0]))
                        if __name__ == "__main__":
                            cut_ind= Parallel(n_jobs=num_cores)(delayed(cutt)(i,datadual11,image_full) for i in inputs)

                    print('take {} seconds in cutting'.format(time.time()-now))
                    if len(cut_ind)>0:
                        cut_ind = np.delete(cut_ind,np.where(np.array(cut_ind)==None)[0])
                        print('cut {} vector'.format(len(cut_ind)))
                        datadual = np.delete(datadual,np.array(cut_ind,dtype='int64'),axis=0)

                if not(ploteach) and not(plot_all): # plot the all data for periodic points together
                    if d2==1:
                        if xz == 1:
                            plt.plot(datadual[:,0],np.arctan2(datadual[:,3],datadual[:,2]),'.',
                                 color=manicolor,markersize=m_size)
                        else:
                            plt.plot(datadual[:,0],datadual[:,1],'.',
                                     color=manicolor,markersize=m_size)
                        if plotimage:
                            plt.plot(image_full[0,:],image_full[1,:],'.',color='black',markersize=m_size)
                    else:
                        if minustheta:
                            plt.plot(datadual[:,0],datadual[:,1],
                                     np.arctan2(datadual[:,3],datadual[:,2])-2*np.pi,
                                     '.',color=manicolor,markersize=m_size)
                        else:
                            y1 = np.where(np.logical_and(datadual[:,1]<0.04,datadual[:,1]>0))[0]
                            x1 = np.where(np.logical_and(datadual[:,0]<1.28, datadual[:,0]>1.215))[0]
                            z1 = np.where(np.logical_and(np.arctan2(datadual[:,3],datadual[:,2])<-2.15,np.arctan2(datadual[:,3],datadual[:,2])>-2.85))[0]
                            plt.plot(datadual[:,0],datadual[:,1],
                                     np.arctan2(datadual[:,3],datadual[:,2]),
                                     '.',color=manicolor,markersize=m_size)
                            if add_minus:
                                plt.plot(datadual[:,0],datadual[:,1],
                                         np.arctan2(datadual[:,3],datadual[:,2])-2*np.pi,
                                         '.',color=manicolor,markersize=m_size)

                if plot_nec_vec:
                    if d2 ==1:
                        if xz == 1:
                            plt.plot(vec[indicator[true_ind],0],np.arctan2(vec[indicator[true_ind],3],vec[indicator[true_ind],2]),'.',color=veccolor,markersize=m_size_vec)
                            plt.plot(vec[indicator_stable,0],np.arctan2(vec[indicator_stable,3],vec[indicator_stable,2]),'g.',markersize=m_size_vec)
                        else:
                            plt.plot(vec[indicator[true_ind],0],vec[indicator[true_ind],1],'.',color=veccolor,markersize=m_size_vec)
                            plt.plot(vec[indicator_stable,0],vec[indicator_stable,1],'g.',markersize=m_size_vec)
                    else:
                        if minustheta:
                            plt.plot(vec[indicator[true_ind],0],vec[indicator[true_ind],1],np.arctan2(vec[indicator[true_ind],3],vec[indicator[true_ind],2])-2*np.pi,'.',color=veccolor,markersize=m_size_vec)
                            ax.plot(vec[indicator_stable,0],vec[indicator_stable,1],np.arctan2(vec[indicator_stable,3],vec[indicator_stable,2])-2*np.pi,'g.',markersize=m_size_vec)
                        else:
                            plt.plot(vec[indicator[true_ind],0],vec[indicator[true_ind],1],np.arctan2(vec[indicator[true_ind],3],vec[indicator[true_ind],2]),'.',color=veccolor,markersize=m_size_vec)
                            plt.plot(vec[indicator_stable,0],vec[indicator_stable,1],np.arctan2(vec[indicator_stable,3],vec[indicator_stable,2]),'g.',markersize=m_size_vec)

                    if plotperiodicity:
                        for iter1 in true_ind:
                            vec_i = vec[indicator[iter1],:]
#                             print(vec_i)
                            pp = int(period[indicator[iter1]])
                            if d2 ==1:
                                if xz == 1:
                                    plt.annotate(str(pp), xy=(vec_i[0], np.arctan2(vec_i[3],vec_i[2])), xytext=(vec_i[0]+5e-4, np.arctan2(vec_i[3],vec_i[2])-5e-4), size=10)
                                else:
                                    if np.any(iter1==dual):
                                        plt.annotate(str(pp), xy=(vec_i[0], vec_i[1]), xytext=(vec_i[0]+5e-4, vec_i[1]-5e-4), size=10)
                                    else:
                                        plt.annotate(str(pp), xy=(vec_i[0], vec_i[1]), xytext=(vec_i[0]-5e-4, vec_i[1]-5e-4), size=10)
                            else:
                                if minustheta:
                                    ax.text(vec_i[0], vec_i[1], np.arctan2(vec_i[3],vec_i[2])-2*np.pi,str(pp), size=10)
                                else:
                                    ax.text(vec_i[0], vec_i[1], np.arctan2(vec_i[3],vec_i[2]),str(pp), size=10)

                        for iter1 in indicator_stable: # stable points
                            vec_i = vec[iter1,:]
#                             print(vec_i)
                            pp = int(period[iter1])
                            if d2 ==1:
                                if xz == 1:
                                    plt.annotate(str(pp), xy=(vec_i[0], np.arctan2(vec_i[3],vec_i[2])), xytext=(vec_i[0]+5e-4, np.arctan2(vec_i[3],vec_i[2])-5e-4), size=10)
                                else:
                                    if target_period == 2:
                                        plt.annotate(str(pp), xy=(vec_i[0], vec_i[1]), xytext=(vec_i[0]+5e-4, vec_i[1]-5e-4), size=10)
                                    else:
                                        plt.annotate(str(pp), xy=(vec_i[0], vec_i[1]), xytext=(vec_i[0]-5e-4, vec_i[1]-5e-4), size=10)
                            else:
                                if minustheta:
                                    ax.text(vec_i[0], vec_i[1], np.arctan2(vec_i[3],vec_i[2])-2*np.pi,str(pp), size=10)
                                else:
                                    ax.text(vec_i[0], vec_i[1], np.arctan2(vec_i[3],vec_i[2]),str(pp), size=10)
                if isdual == 0:
                    if len(boundary) == 0:
                        boundary = datadual
                    else:
                        boundary = np.insert(boundary,np.shape(boundary)[0],datadual,axis=0)
                else:
                    if len(boundary_dual) == 0:
                        boundary_dual = datadual
                    else:
                        boundary_dual = np.insert(boundary_dual,np.shape(boundary_dual)[0],datadual,axis=0)
                        
        if plot_all:
            if save_work and clear_interior:
                a_sub = np.round(a_start+jj/nn*total_plus,5)
                filename = 'ori_a'+str(a_sub)+'.npy'
                with open(filename, 'wb') as f:
                    np.save(f, boundary)
                filename = 'dual_ori_a'+str(a_sub)+'.npy'
                with open(filename, 'wb') as f:
                    np.save(f, boundary_dual)
            # compute the image
            if period_tgt:
                if len(boundary) != 0:
                    image_full = ff(boundary.transpose())
                if len(boundary_dual) != 0:
                    image_full_dual = ff(boundary_dual.transpose())
                    datadual11 = F(boundary_dual.transpose()).transpose()
            else:
                if len(boundary_dual) != 0:
                    datadual11 = F(boundary_dual.transpose()).transpose()
                    image_full_dual = ff(boundary_dual.transpose())
                if len(boundary) != 0:
                    datadual111 = F_n(boundary.transpose(),target_period-1).transpose()
                    image_full = ff(datadual111.transpose())
            # only display boundary for mis or dual
            if clear_interior:
                now=time.time()
                if len(boundary) != 0:
                    def cutt(datvec,datadual,image_full):
                        if np.any(np.linalg.norm(datadual[datvec,:2]-image_full.transpose(),axis=1)<epsilon-epsilon_tol):
                            return datvec
                    print('cutting')
                    inputs = tqdm(range(np.shape(boundary)[0]))
                    if __name__ == "__main__":
                        cut_ind= Parallel(n_jobs=num_cores)(delayed(cutt)(i,boundary,image_full) for i in inputs)
                    print('take {} seconds in cutting'.format(time.time()-now))
                    if len(cut_ind)>0:
                        cut_ind = np.delete(cut_ind,np.where(np.array(cut_ind)==None)[0])
                        print('cut {} vector'.format(len(cut_ind)))
                        boundary = np.delete(boundary,np.array(cut_ind,dtype='int64'),axis=0)        
                if len(boundary_dual) != 0:
                    def cutt(datvec,datadual,image_full):
                        if np.any(np.linalg.norm(datadual[:,:2]-image_full[:,datvec].transpose(),axis=1)<epsilon-epsilon_tol):
                            return datvec
                    print('cutting')
                    inputs = tqdm(range(np.shape(boundary_dual)[0]))
                    if __name__ == "__main__":
                        cut_ind= Parallel(n_jobs=num_cores)(delayed(cutt)(i,datadual11,image_full_dual) for i in inputs)
                    print('take {} seconds in cutting'.format(time.time()-now))
                    if len(cut_ind)>0:
                        cut_ind = np.delete(cut_ind,np.where(np.array(cut_ind)==None)[0])
                        print('cut {} vector'.format(len(cut_ind)))
                        boundary_dual = np.delete(boundary_dual,np.array(cut_ind,dtype='int64'),axis=0)        
            if d2==1:
                if xz == 1:
                    if len(boundary) != 0:
                        plt.plot(boundary[:,0],np.arctan2(boundary[:,3],boundary[:,2]),'.',
                             color=manicolor1,markersize=m_size)
                    if len(boundary_dual) != 0:
                        plt.plot(boundary_dual[:,0],np.arctan2(boundary_dual[:,3],boundary_dual[:,2]),'.',
                             color=manicolor_dual,markersize=m_size)
                else:
                    if len(boundary) != 0:
                        plt.plot(boundary[:,0],boundary[:,1],'.',
                                 color=manicolor1,markersize=m_size)
                    if len(boundary_dual) != 0:
                        plt.plot(boundary_dual[:,0],boundary_dual[:,1],'.',
                                 color=manicolor_dual,markersize=m_size)
            if save_work:
                a_sub = np.round(a_start+jj/nn*total_plus,5)
                filename = 'a'+str(a_sub)+'.npy'
                with open(filename, 'wb') as f:
                    np.save(f, boundary)
                filename = 'unstable_a'+str(a_sub)+'.npy'
                with open(filename, 'wb') as f:
                    np.save(f,vec[[i for i in indicator if all(i != indicator[dual])],:])
                filename = 'stable_a'+str(a_sub)+'.npy'
                with open(filename, 'wb') as f:
                    np.save(f,vec[indicator_stable,:])
                filename = 'dual_a'+str(a_sub)+'.npy'
                with open(filename, 'wb') as f:
                    np.save(f, boundary_dual)
                filename = 'dual_point_a'+str(a_sub)+'.npy'
                with open(filename, 'wb') as f:
                    np.save(f,vec[indicator[dual],:])

    else:
        data = manifold_all[str(jj)]
        for ii in range(len(data)):
            if np.any(ii==dual): # dual repeller
                print(ii)
                datadual = data[ii][0]
                datadual = np.delete(datadual,np.unique(np.where(np.abs(datadual)>limit)[0]),axis=0)
                datadual = np.delete(datadual,np.unique(np.where(np.isnan(datadual))[0]),axis=0)
                print('done')
                if d2 == 1: # 2-dimension
                    plt.plot(datadual[:,0],datadual[:,1],'.',color='orange',markersize=m_size)
                else: # 3-dimension
                    ax.plot(datadual[:,0],datadual[:,1],np.arctan2(datadual[:,3],datadual[:,2]),'.',color='orange',markersize=m_size)
                if plotnext: # plot one step next
                    Ft = F_invn(np.transpose(datadual),1);
                    if d2==1:
                        plt.plot(Ft[0,:],Ft[1,:],'.',color='cyan',markersize=m_size)
                    else:
                        plt.plot(Ft[0,:],Ft[1,:],np.arctan2(Ft[3,:],Ft[2,:]),'.',color='black',markersize=m_size)
                if plotimage: # plot image of projection of stable manifold
                    ft = f(np.transpose(datadual))
                    if d2 == 1:
                        plt.plot(ft[0,:],ft[1,:],'.',color='cyan',markersize=1)
                    else:
                        ax.plot(ft[0,:],ft[1,:],np.arctan2(ft[3,:],ft[2,:]),'.',color='cyan',markersize=1)
#                 continue
            else:
                if d2 == 1:
                    plt.plot(data[ii][0][:,0],data[ii][0][:,1],'.',color='#1f77b4',markersize=m_size)
                else:
                    ax.plot(data[ii][0][:,0],data[ii][0][:,1],np.arctan2(data[ii][0][:,3],data[ii][0][:,2]),'.',color='#1f77b4',markersize=m_size)
                if plotimage: # plot image of projection of unstable manifold
                    ft = f(np.transpose(data[ii][0]))
                    if d2==1:
                        plt.plot(ft[0,:],ft[1,:],'.',color='cyan',markersize=m_size)
                    else:
                        plt.plot(ft[0,:],ft[1,:],np.arctan2(ft[3,:],ft[2,:]),'.',color='cyan',markersize=m_size)

    # plot stable and unstable periodic points
    if plotvec and not(plot_nec_vec):
        if d2 == 1:
            if xz == 1:
                plt.plot(vec[indicator,0],np.arctan2(vec[indicator,3],vec[indicator,2]),'r.',markersize=m_size_vec)
                plt.plot(vec[indicator_stable,0],np.arctan2(vec[indicator_stable,3],vec[indicator_stable,2]),'g.',markersize=m_size_vec)
                plt.plot(vec[dual,0],np.arctan2(vec[dual,3],vec[dual,2]),'.',color='purple',markersize=m_size_vec)
                plt.plot(vec[indicator_unstable,0],np.arctan2(vec[indicator_unstable,3],vec[indicator_unstable,2]),'.',markersize=m_size_vec,color='blue')
            else:
                plt.plot(vec[indicator,0],vec[indicator,1],'r.',markersize=m_size_vec)
                plt.plot(vec[indicator_stable,0],vec[indicator_stable,1],'g.',markersize=m_size_vec)
                plt.plot(vec[dual,0],vec[dual,1],'.',color='purple',markersize=m_size_vec)
                plt.plot(vec[indicator_unstable,0],vec[indicator_unstable,1],'.',markersize=m_size_vec,color='blue')
        else:
            ax.plot(vec[indicator,0],vec[indicator,1],np.arctan2(vec[indicator,3],vec[indicator,2]),'r.',markersize=m_size_vec)
            ax.plot(vec[indicator_stable,0],vec[indicator_stable,1],np.arctan2(vec[indicator_stable,3],vec[indicator_stable,2]),'g.',markersize=m_size_vec)
            ax.plot(vec[dual,0],vec[dual,1],np.arctan2(vec[dual,3],vec[dual,2]),'.',color='purple',markersize=m_size_vec)
            plt.plot(vec[indicator_unstable,0],vec[indicator_unstable,1],np.arctan2(vec[indicator_unstable,3],vec[indicator_unstable,2]),'.',markersize=m_size_vec,color='blue')

        # plot the periodicity
        if plotperiodicity:
            for i in range(np.shape(vec)[0]):
                vec_i = vec[i,:]
                pp = int(period[i])
                if d2 == 1:
                    if xz == 1:
                        plt.annotate(str(pp), xy=(vec_i[0],  np.arctan2(vec_i[3],vec_i[2])), xytext=(vec_i[0]-5e-4, np.arctan2(vec_i[3],vec_i[2])-5e-4), size=10)
                    else:
                        if target_period == 2:
                            plt.annotate(str(pp), xy=(vec_i[0], vec_i[1]), xytext=(vec_i[0]-5e-4, vec_i[1]-5e-4), size=10)
                        else:
                            plt.annotate(str(pp), xy=(vec_i[0], vec_i[1]), xytext=(vec_i[0]-5e-1, vec_i[1]-5e-1), size=10)
                else:
                    ax.text(vec_i[0], vec_i[1], np.arctan2(vec_i[3],vec_i[2]),str(pp), size=10)    
    print(time.time()-start_time)

    if d2==1:
        if xz == 1:
            plt.xlabel('x')
            plt.ylabel('theta')
            plt.title('a = '+str(a_start+jj/nn*total_plus)+' b = '+str(b_sub)+' eps = '+str(eps_start))
        else:
            plt.axis('equal')
            plt.xlim([-0.5,2.])
            plt.ylim([-1.,1.25])
            plt.xlim([-1.5,3.5])
            plt.ylim([-3,3])
            plt.xlabel('x')
            plt.ylabel('y')
            plt.title('a = '+str(a_start+jj/nn*total_plus)+' b = '+str(b_sub)+' eps = '+str(eps_start))
    #         plt.title('a = '+str(a_start)+' b = '+str(b_sub)+' eps = '+str(eps_start+jj/nn*total_plus)) 
    #         plt.savefig('henon_'+str(a_start+jj/nn*total_plus)+'stable_and_strongunstable.png',dpi=300) # save the figure
    else:

#         ax.set_xlim([0.6,1.8])
#         ax.set_ylim([-0.5,0.4])
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('theta')
        ax.set_title('a = '+str(a_start+jj/nn*total_plus)+' b = '+str(b_sub)+' eps = '+str(eps_start))
    plt.show()
    print(a_start+jj/nn*total_plus)
    #     ax.view_init(azim=-54, elev=12) # view of the 3-D plot

2.7600088119506836
1.12


In [14]:
# plt.savefig('henon_'+str(a_start+jj/nn*total_plus)+'.png',dpi=300) # save the figure

In [54]:
for jj in range(294):
    dual = dual_all[str(jj)]
    vec = vec_all[str(jj)]
    indicator = indicator_all[str(jj)]
    period = period_all[str(jj)]
    indicator_stable = indicator_stable_all[str(jj)]
    a_sub = np.round(a_start+jj/nn*total_plus,5)
    plt.plot(vec[indicator,0],vec[indicator,1],'r.')
    filename = 'unstable_a'+str(a_sub)+'.npy'
#     find_ind = []
#     for i in indicator:
#         if (period[i] == 2 and all(indicator[dual] != i)):
#             find_ind.append(i)
#     print(find_ind)
    with open(filename, 'wb') as f:
        np.save(f,vec[[i for i in indicator if all(i != indicator[dual])],:])
    if len(indicator_stable) != 0:
        plt.plot(vec[indicator_stable,0],vec[indicator_stable,1],'g.')
        filename = 'stable_a'+str(a_sub)+'.npy'
        with open(filename, 'wb') as f:
            np.save(f,vec[indicator_stable,:])
    filename = 'dual_point_a'+str(a_sub)+'.npy'
#     find_ind = []
#     for i in indicator:
#         if (period[i] == 2 and any(indicator[dual] == i)):
#             find_ind.append(i)
    with open(filename, 'wb') as f:
        np.save(f,vec[indicator[dual],:])
    plt.xlim([-1.5,3.5])
    plt.ylim([-3,3])
    plt.title('a = '+str(a_start+jj/nn*total_plus)+' b = '+str(b_sub)+' eps = '+str(eps_start))
    plt.show()
    
    
    

In [71]:
# plt.savefig('zoom_henon_'+str(a_start+jj/nn*total_plus)+'.png',dpi=300) # save the figure
plt.plot(vec[indicator,0],vec[indicator,1],'r.')
plt.plot(vec[indicator[dual],0],vec[indicator[dual],1],'k.')
plt.plot(vec[indicator_stable,0],vec[indicator_stable,1],'b.')
print(vec)

[[-1.04763743  0.38705084  0.81172008  0.58404667]
 [ 0.04078119 -0.31429123  0.2463428  -0.96918276]
 [ 0.68366877  0.01223436 -0.88947578  0.45698231]
 [ 0.4389085   0.20510063  0.74427001 -0.66787884]
 [ 0.96880373  0.13167255 -0.9922245   0.12446101]
 [-0.01960927  0.29064112  0.05353797 -0.99856581]
 [ 1.29016946 -0.00588278 -0.94704073  0.32111347]
 [ 0.68366877  0.01223436  0.88947578 -0.45698231]
 [ 0.4389085   0.20510063 -0.74427001  0.66787884]
 [ 0.96880373  0.13167255  0.9922245  -0.12446101]
 [-0.01960927  0.29064112 -0.05353797  0.99856581]
 [ 1.29016946 -0.00588278  0.94704073 -0.32111347]
 [-1.04763743  0.38705084 -0.81172008 -0.58404667]
 [ 0.04078119 -0.31429123 -0.2463428   0.96918276]
 [ 1.29004566 -0.00532843 -0.94715487  0.32077665]
 [-1.04669127  0.3870137   0.81653901  0.57729026]
 [ 0.04317467 -0.31400738  0.25173507 -0.96779618]
 [ 0.68370613  0.0129524  -0.88939857  0.45713256]
 [ 0.43956388  0.20511184  0.7453809  -0.66663882]
 [ 0.96810873  0.13186916 -0.99

In [63]:
# Numerical continuation for different parameter of a_sub
# redefine the starting parameter
a_sub, b_sub, epsilon = a_start, b_start, eps_start
nn = 100 # number of parameter to run
searchtol = 1e-9
total_plus = -0.003 # maximum addition to parameter
d = 2
draw_mani = 0
near_tol = 1e-0 # only find periodic point sufficiently close to the original

for jj in range(1,nn+1):
    start_time = time.time()
    a_sub += total_plus/nn # increase size of parameter
#     epsilon += total_plus/nn # increase epsilon
#     b_sub += total_plus/nn # increase b_sub
    print(jj)
    
    # Deterministic steady point
    n = 2
    nsqr = n**2
    xt, yt = np.linspace(-10,10,n), np.linspace(-10,10,n)
    xx, yy = np.meshgrid(xt,yt)
    xx, yy = np.reshape(xx,nsqr), np.reshape(yy,nsqr)
    ll = np.unique(np.round(ff_n(np.array([xx,yy]),1000),4),axis=1)
    ll = ll[:,np.unique(np.where(np.isfinite(ll))[1])]
    vec_det_all[str(jj)] = ll
    
    n = 10 # number of direction and nearby points per dimension to search
    vec1 = [] # initialization of the periodic points
    period_new = [] # record period of vec
    for ii in range(np.shape(vec)[0]):
        n_period = int(period[ii])
        F_periodeq = lambda xx: F_n(xx,n_period)-xx
        Jac_periodeq = lambda xx: Jac_eqn(xx,n_period)
        vec1,period_new = find_period_continue(vec,vec1,n,tolerance,period_new,n_period,ii,near_tol)
        #period_new = np.append(period_new,np.ones(np.shape(vec1)[0]-len(period_new))*n_period) 
        if (np.any(np.abs(np.abs(eigenstack[ii])-1)<1e-1)) and (not(np.any(period-n_period*2==0))):
            print('detect period doubling')
            n_period = int(n_period*2)
            F_periodeq = lambda xx: F_n(xx,n_period)-xx
            Jac_periodeq = lambda xx: Jac_eqn(xx,n_period)
            vec1,period_new = find_period_continue(vec,vec1,n,tolerance,period_new,n_period,ii,near_tol)
    period = np.array(period_new)
    vec = np.array(vec1)
    end_time = time.time()
    print('elapsed time after searching periodic points: {}s at a = {}, b= {}, eps = {}'\
          .format(np.round(end_time-start_time,2), a_sub, b_sub, epsilon))
    
    # getting rid points not on compact invariant set
    if compact == 1:
        delete_ind = np.array([])
        inputs = tqdm(range(np.shape(vec)[0]))
        if __name__ == "__main__":
            processed_list = Parallel(n_jobs=num_cores)(delayed(compacting)(i,vec) for i in inputs)
        delete_ind = [i for i in processed_list if i !=None]
        if len(delete_ind)>0:
            vec = np.delete(vec,np.int64(delete_ind),0)
            period = np.delete(period,np.int64(delete_ind))

    # record eigenvalues
    eigenstack_det = np.zeros((np.shape(vec)[0],d),dtype=complex)
    eigenstack = np.zeros((np.shape(vec)[0],2*d-1),dtype=complex)
    eigenvecstack = np.zeros((np.shape(vec)[0],2*d,2*d-1),dtype=complex)
    for i in range(np.shape(vec)[0]):
        n_period = int(period[i])
        Jac_period = lambda xx: Jac_eqn(xx,n_period)+np.eye(4)
        Jac_period_det = lambda xx: Jac_detn(xx,n_period)
        #print(Jac_period(vec[i,:])) # for debug
        eigen, eigenvec = sl.eig(Jac_period(vec[i,:]))
        #print(all(np.real(eigen))) # for debug
        eigen_det = sl.eigvals(Jac_period_det(vec[i,:]))
        pts = vec[i,:]
        # remove zero eigenvalue (reduce dimension to 2d-1)
        remove_ind = np.where(np.abs(eigen)<1e-5)[0]
        if len(remove_ind)==0:
            print('no_zero')
            remove_ind = 2*d-1
        eigenstack_det[i,:]=eigen_det
        if len(remove_ind)>1: # one eigenvalue very small near 0
            print('more than one 0 eigenvalues')
            remove_ind = 3
        eigenstack[i,:]=np.delete(eigen,remove_ind)
        eigenvecstack[i,:,:]=np.delete(eigenvec,remove_ind,1)
        #print('eigenvalues is [{:.2f},{:.2f},{:.2f}] at [{:.2f},{:.2f},{:.2f}]'
        #      .format(eigen[0],eigen[1],eigen[2],pts[0],pts[1],pts[2])) # for debug
    
    # remove points with two or more unstable eigenspace
    if dualoff:
        delete_ind = np.array([])
        for i in range(np.shape(vec)[0]):
            count = 0
            for m in range(np.size(eigenstack[i,:])):
                if np.abs(eigenstack[i,m])>1:
                    count += 1
            if count > 1:
                delete_ind=np.insert(delete_ind,0,i)
    
        vec = np.delete(vec,np.int64(delete_ind),0)
        eigenstack = np.delete(eigenstack,np.int64(delete_ind),0)
        eigenvecstack = np.delete(eigenvecstack,np.int64(delete_ind),0)
        period = np.delete(period,np.int64(delete_ind),0)

    vec_all[str(jj)] = np.array(vec) # define fist periodic points
    eigen_all[str(jj)] = np.array(eigenstack)
    eigenvec_all[str(jj)] = np.array(eigenvecstack)
    eigen_det_all[str(jj)] = np.array(eigenstack_det)
    period_all[str(jj)] = np.array(period)
    print('jj = {}'.format(jj))
    print('size of the critical point:{}'.format(np.shape(vec)[0]))
    print('periodic points: \n{}'.format(np.round(np.transpose(vec),2)))
    print('eigen: \n{}'.format(np.round(np.transpose(eigenstack),3)))
    print('period: \n{}'.format(period))
    #print('eigen_deterministic: \n{}'.format(np.round(eigenstack_det,3))) # for debug
    #print(np.round(eigenvecstack,3)) # for debug
     
    # determine periodic points
    lenvec = np.shape(vec)[0]
    record_vec = [0]
    for counter in range(lenvec-1): # record start of periodic point
        if np.linalg.norm(F(vec[counter,:])-vec[counter+1,:])>1e-3:
            record_vec.append(counter+1)
    record_all[str(jj)] = np.array(record_vec)
    
    # determine stable and unstable points
    dual = [] # indices of one stable
    indicator_unstable = [] # indices of all unstable
    indicator_stable = [] # indices of all stable
    indicator = [] # indices of one unstable
    eigensearch = [] # eigenvector for direction of search
    i = 0
    while i < np.shape(vec)[0]:
        eigenhere = np.array(eigenstack[i])
        eigenvechere = np.array(eigenvecstack[i])
        if all(np.abs(eigenhere)>1): # all eigen diverge
            indicator_unstable.append(i)
            i += 1
            continue
        elif all(np.abs(eigenhere)<1): # all eigen converge
            indicator_stable.append(i)
            i += 1
            continue
        choice = np.argwhere(np.abs(eigenhere)>=1)
        if len(choice)>d-1: # more than d-1 unstable direction which must be on dual since 
                            #in dim d, all points on boundary attracts outer points
            target=eigenvechere[:,np.argwhere(np.abs(eigenhere)<1)[0][0]]
            dual.append(len(indicator))
        else:
            target=eigenvechere[:,choice[0][0]] # choose the unstable eigenvalue direction
        eigensearch.append(target)
        indicator.append(i)
        i += 1
        
    eigensearch = np.real(np.array(eigensearch))
    indicator = np.array(indicator)
    dual = np.array(dual)
    if np.size(indicator) == 0:
        print('no unstable points to approximate')
        continue
    
    indicator_all[str(jj)] = indicator
    indicator_stable_all[str(jj)] = indicator_stable
    indicator_unstable_all[str(jj)] = indicator_unstable
    dual_all[str(jj)] = dual
    end_time = time.time()
    print('elapsed time before unstable manifold calc: {}s'.format(np.round(end_time-start_time,2)))

    # find invariant manifold
    if draw_mani:
        if not(brute):
            manifold_all[str(jj)]= unstable_mani(vec,indicator,eigensearch,dual,eigenstack,period)
        else:
            manifold_all[str(jj)] = unstable_mani_brute(vec,indicator,eigensearch,dual,eigenstack,period)
        #,traj_each_all[str(jj)]
    end_time = time.time()
    print('elapsed time in unstable manifold calc: {}s'.format(np.round(end_time-start_time,2)))

1
elapsed time after searching periodic points: 1.7s at a = 1.22797, b= 0.3, eps = 0.0


100%|████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 9722.54it/s]

jj = 1
size of the critical point:30
periodic points: 
[[ 0.01  0.68  0.43  0.98 -0.04  1.29 -1.06  0.66  0.07  0.68  0.45  0.96
   0.    1.29 -1.04  0.43  0.98 -0.04  1.29 -1.06  0.01  0.68  0.66  0.07
   0.68  0.45  0.96  0.    1.29 -1.04]
 [-0.32  0.    0.2   0.13  0.29 -0.01  0.39  0.2  -0.31  0.02  0.21  0.13
   0.29  0.    0.39  0.2   0.13  0.29 -0.01  0.39 -0.32  0.    0.2  -0.31
   0.02  0.21  0.13  0.29  0.    0.39]
 [-0.18  0.89 -0.73  1.   -0.01  0.94 -0.71  0.17 -0.31  0.89 -0.76  0.98
  -0.11  0.95 -0.85  0.73 -1.    0.01 -0.94  0.71  0.18 -0.89 -0.17  0.31
  -0.89  0.76 -0.98  0.11 -0.95  0.85]
 [ 0.98 -0.46  0.68 -0.03  1.   -0.33 -0.71  0.99  0.95 -0.46  0.65 -0.19
   0.99 -0.32 -0.53 -0.68  0.03 -1.    0.33  0.71 -0.98  0.46 -0.99 -0.95
   0.46 -0.65  0.19 -0.99  0.32  0.53]]
eigen: 
[[  -598.47 +0.j   -598.47 +0.j   -598.47 +0.j   -598.47 +0.j
    -598.47 +0.j   -598.47 +0.j   -598.47 +0.j     -1.792+0.j
  -16981.572+0.j -16981.572+0.j -16981.572+0.j -16981.572+0.j
  

elapsed time before unstable manifold calc: 1.91s
elapsed time in unstable manifold calc: 1.91s
2
elapsed time after searching periodic points: 1.71s at a = 1.22794, b= 0.3, eps = 0.0


100%|███████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 24720.85it/s]

...

100%|████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]

jj = 99
size of the critical point:2
periodic points: 
[[ 0.66  0.66]
 [ 0.2   0.2 ]
 [ 0.17 -0.17]
 [ 0.99 -0.99]]
eigen: 
[[-1.789+0.j -1.789+0.j]
 [ 0.168+0.j  0.168+0.j]
 [-0.094+0.j -0.094+0.j]]
period: 
[1. 1.]
elapsed time before unstable manifold calc: 0.03s
elapsed time in unstable manifold calc: 0.03s
100


elapsed time after searching periodic points: 0.01s at a = 1.2250000000000025, b= 0.3, eps = 0.0


100%|██████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1999.19it/s]

jj = 100
size of the critical point:2
periodic points: 
[[ 0.66  0.66]
 [ 0.2   0.2 ]
 [ 0.17 -0.17]
 [ 0.99 -0.99]]
eigen: 
[[-1.789+0.j -1.789+0.j]
 [ 0.168+0.j  0.168+0.j]
 [-0.094+0.j -0.094+0.j]]
period: 
[1. 1.]
elapsed time before unstable manifold calc: 0.03s
elapsed time in unstable manifold calc: 0.03s
